In [2]:
import zipfile

# Unzip the dataset
zip_path = "/content/archive.zip"
extract_path = "/content/dataset"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


In [3]:
import os

# List files to see what's inside
for root, dirs, files in os.walk(extract_path):
    for file in files:
        print(os.path.join(root, file))


/content/dataset/dairy_dataset.csv


In [4]:
import pandas as pd

# Load the dataset
df = pd.read_csv('/content/dataset/dairy_dataset.csv')

# Preview the dataset
print(df.shape)
print(df.columns)
print(df.head())

# Basic info
df.info()
df.describe()


(4325, 23)
Index(['Location', 'Total Land Area (acres)', 'Number of Cows', 'Farm Size',
       'Date', 'Product ID', 'Product Name', 'Brand', 'Quantity (liters/kg)',
       'Price per Unit', 'Total Value', 'Shelf Life (days)',
       'Storage Condition', 'Production Date', 'Expiration Date',
       'Quantity Sold (liters/kg)', 'Price per Unit (sold)',
       'Approx. Total Revenue(INR)', 'Customer Location', 'Sales Channel',
       'Quantity in Stock (liters/kg)', 'Minimum Stock Threshold (liters/kg)',
       'Reorder Quantity (liters/kg)'],
      dtype='object')
        Location  Total Land Area (acres)  Number of Cows Farm Size  \
0      Telangana                   310.84              96    Medium   
1  Uttar Pradesh                    19.19              44     Large   
2     Tamil Nadu                   581.69              24    Medium   
3      Telangana                   908.00              89     Small   
4    Maharashtra                   861.95              21    Medium   

   

,Total Land Area (acres),Number of Cows,Product ID,Quantity (liters/kg),Price per Unit,Total Value,Shelf Life (days),Quantity Sold (liters/kg),Price per Unit (sold),Approx. Total Revenue(INR),Quantity in Stock (liters/kg),Minimum Stock Threshold (liters/kg),Reorder Quantity (liters/kg)
count,4325.000000,4325.000000,4325.000000,4325.000000,4325.000000,4325.000000,4325.000000,4325.000000,4325.00000,4325.000000,4325.000000,4325.000000,4325.000000
mean,503.483073,54.963699,5.509595,500.652657,54.785938,27357.845411,29.127630,248.095029,54.77914,13580.265401,252.068671,55.826143,109.107820
std,285.935061,26.111487,2.842979,288.975915,26.002815,21621.051594,30.272114,217.024182,26.19279,14617.009122,223.620870,26.301450,51.501035
min,10.170000,10.000000,1.000000,1.170000,10.030000,42.516500,1.000000,1.000000,5.21000,12.540000,0.000000,10.020000,20.020000
25%,252.950000,32.000000,3.000000,254.170000,32.460000,9946.814500,10.000000,69.000000,32.64000,2916.650000,66.000000,32.910000,64.280000
50%,509.170000,55.000000,6.000000,497.550000,54.400000,21869.652900,22.000000,189.000000,54.14000,8394.540000,191.000000,56.460000,108.340000
75%,751.250000,77.000000,8.000000,749.780000,77.460000,40954.441000,30.000000,374.000000,77.46000,19504.550000,387.000000,79.010000,153.390000
max,999.530000,100.000000,10.000000,999.930000,99.990000,99036.369600,150.000000,960.000000,104.51000,89108.900000,976.000000,99.990000,199.950000


In [5]:
# Handle missing values
df = df.dropna()  # Or use imputation if needed

# Parse date columns if available
df['Date'] = pd.to_datetime(df['Date'])
df['Production Date'] = pd.to_datetime(df['Production Date'])
df['Expiration Date'] = pd.to_datetime(df['Expiration Date'])

# Feature: Days Until Expiry
df['Days_Until_Expiry'] = (df['Expiration Date'] - df['Date']).dt.days

# Drop rows where expiry is negative (expired stock)
df = df[df['Days_Until_Expiry'] >= 0]


In [12]:
print(df.columns.tolist())


['Location', 'Total Land Area (acres)', 'Number of Cows', 'Farm Size', 'Date', 'Product ID', 'Product Name', 'Brand', 'Quantity (liters/kg)', 'Price per Unit', 'Total Value', 'Shelf Life (days)', 'Storage Condition', 'Production Date', 'Expiration Date', 'Quantity Sold (liters/kg)', 'Price per Unit (sold)', 'Approx. Total Revenue(INR)', 'Customer Location', 'Sales Channel', 'Quantity in Stock (liters/kg)', 'Minimum Stock Threshold (liters/kg)', 'Reorder Quantity (liters/kg)', 'Days_Until_Expiry', 'Stock_to_Sales_Ratio']


In [13]:
# Rename for convenience (optional)
df = df.rename(columns={
    'Quantity in Stock (liters/kg)': 'quantity_in_stock',
    'Quantity Sold (liters/kg)': 'quantity_sold',
    'Reorder Quantity (liters/kg)': 'reorder_quantity',
    'Minimum Stock Threshold (liters/kg)': 'min_stock_threshold'
})

# Derived Feature: Demand Velocity
df['demand_velocity'] = df['quantity_sold'] / df['Days_Until_Expiry'].replace(0, 1)

# Target Variables
df['days_to_reorder'] = (df['min_stock_threshold'] - df['quantity_in_stock']) / df['demand_velocity']
df['days_to_reorder'] = df['days_to_reorder'].clip(lower=0)


In [14]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import numpy as np

# Encode categorical features
categorical_cols = ['Location', 'Product ID', 'Brand', 'Storage Condition']
label_encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# Select features
numerical_cols = ['quantity_in_stock', 'quantity_sold', 'Shelf Life (days)', 'Days_Until_Expiry',
                  'Stock_to_Sales_Ratio', 'demand_velocity']

feature_cols = categorical_cols + numerical_cols

# Scale numerical data
scaler = MinMaxScaler()
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

# Inputs and outputs
X = df[feature_cols].values
y1 = df['reorder_quantity'].values  # Output 1
y2 = df['days_to_reorder'].values   # Output 2

# Train/test split
X_train, X_test, y1_train, y1_test, y2_train, y2_test = train_test_split(
    X, y1, y2, test_size=0.2, random_state=42
)


In [15]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

# Define input layer
input_layer = Input(shape=(X.shape[1],))

# Shared hidden layers
x = Dense(128, activation='relu')(input_layer)
x = Dense(64, activation='relu')(x)

# Output layers
reorder_output = Dense(1, name='reorder_quantity')(x)
days_output = Dense(1, name='days_to_reorder')(x)

# Model
model = Model(inputs=input_layer, outputs=[reorder_output, days_output])

# Compile
model.compile(
    optimizer='adam',
    loss={'reorder_quantity': 'mse', 'days_to_reorder': 'mse'},
    metrics={'reorder_quantity': 'mae', 'days_to_reorder': 'mae'}
)

model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 10)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │      1,408 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 64)        │      8,256 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reorder_quantity    │ (None, 1)         │         65 │ dense_1[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ days_to_reorder     │ (None, 1)         │         65 │ dense_1[0][0]     │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 9,794 (38.26 KB)

 Trainable params: 9,794 (38.26 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
history = model.fit(
    X_train, {'reorder_quantity': y1_train, 'days_to_reorder': y2_train},
    validation_data=(X_test, {'reorder_quantity': y1_test, 'days_to_reorder': y2_test}),
    epochs=5,
    batch_size=32
)


Epoch 1/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - days_to_reorder_loss: 76557.8125 - days_to_reorder_mae: 25.9227 - loss: 89787.9219 - reorder_quantity_loss: 13230.0967 - reorder_quantity_mae: 102.8428 - val_days_to_reorder_loss: 77725.5391 - val_days_to_reorder_mae: 25.4883 - val_loss: 93187.2266 - val_reorder_quantity_loss: 7940.2075 - val_reorder_quantity_mae: 77.6217
Epoch 2/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - days_to_reorder_loss: 15717.2627 - days_to_reorder_mae: 20.5563 - loss: 22240.4277 - reorder_quantity_loss: 6523.1646 - reorder_quantity_mae: 65.9188 - val_days_to_reorder_loss: 77566.4453 - val_days_to_reorder_mae: 37.3594 - val_loss: 88445.7734 - val_reorder_quantity_loss: 3999.5879 - val_reorder_quantity_mae: 53.0960
Epoch 3/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - days_to_reorder_loss: 42878.7188 - days_to_reorder_mae: 30.3736 - loss: 46544.4570 - reorder_quantity_loss: 3665.7493 - reorder_quantity_mae: 50.1982 - val_days_to_reorder_loss: 77544.4141 - val_days_

In [17]:
model.evaluate(X_test, {'reorder_quantity': y1_test, 'days_to_reorder': y2_test})


12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - days_to_reorder_loss: 154124.3281 - days_to_reorder_mae: 45.5871 - loss: 158682.9375 - reorder_quantity_loss: 3508.5955 - reorder_quantity_mae: 50.2252


[87646.859375,
 3409.5419921875,
 77412.15625,
 33.70298385620117,
 49.67924880981445]

In [19]:
# Get predicted values
y_pred_quantity, y_pred_days = model.predict(X_test)

# Convert to 1D
y_pred_quantity = y_pred_quantity.flatten()
y_pred_days = y_pred_days.flatten()


12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step


In [21]:
# Use inverse_transform if you label-encoded these columns
location_decoded = label_encoders['Location'].inverse_transform(X_test[:, feature_cols.index('Location')].astype(int))
product_id_decoded = label_encoders['Product ID'].inverse_transform(X_test[:, feature_cols.index('Product ID')].astype(int))


In [22]:
import pandas as pd

# Build final prediction table
pred_df = pd.DataFrame({
    'Location': location_decoded,
    'Product ID': product_id_decoded,
    'Days to Reorder': y_pred_days.round(0).astype(int),
    'Reorder Quantity': y_pred_quantity.round(1)
})

# Display top 10
print(pred_df.head(10))


         Location  Product ID  Days to Reorder  Reorder Quantity
0       Telangana          10               19        139.600006
1  Madhya Pradesh           1               14        106.099998
2          Kerala           1               13        120.199997
3      Chandigarh          10               18         88.500000
4      Tamil Nadu          10               20        143.399994
5   Uttar Pradesh           2               13         96.599998
6          Kerala           2               11         64.000000
7      Tamil Nadu           3               15        122.800003
8       Telangana           9               22        166.899994
9         Gujarat           3               13         78.300003


In [24]:
pred_df

,Location,Product ID,Days to Reorder,Reorder Quantity
0,Telangana,10,19,139.600006
1,Madhya Pradesh,1,14,106.099998
2,Kerala,1,13,120.199997
3,Chandigarh,10,18,88.500000
4,Tamil Nadu,10,20,143.399994
...,...,...,...,...
348,Haryana,2,12,75.000000
349,Karnataka,5,12,74.699997
350,Rajasthan,5,15,85.800003
351,Telangana,2,14,80.000000


In [25]:
# Assuming your DataFrame is named pred_df
grouped_df = pred_df.groupby(['Location', 'Product ID']).agg({
    'Days to Reorder': 'mean',
    'Reorder Quantity': 'mean'
}).reset_index()


In [26]:
print(grouped_df)

        Location  Product ID  Days to Reorder  Reorder Quantity
0          Bihar           1        11.000000         69.400002
1          Bihar           2         7.500000         58.500000
2          Bihar           3         9.333333         68.900002
3          Bihar           4         8.500000         57.199997
4          Bihar           5        11.000000         61.500000
..           ...         ...              ...               ...
110  West Bengal           4        18.000000        128.100006
111  West Bengal           5        18.000000        105.300003
112  West Bengal           6        21.000000        140.399994
113  West Bengal           7        24.000000        140.699997
114  West Bengal          10        22.000000        137.519989

[115 rows x 4 columns]
